# Attempting to Model Elevator Situation with SimPy
## The Problem
My apartment building has three elevators, but one is reserved for freight. So, it really has two elevators which are called with one button. 

One of the elevators broke down, so the building opened the freight elevator up to the residents. The freight elevator has its own button on each floor.

I noticed one day as I was descending that my elevator kept stopping and no one would get in. The people in my building had started pressing both buttons and then getting in the elevator that arrived first.

That got me thinking about math. Here are my research questions:
1. Are we all worse off if everyone does this behavior?
2. Is there a threshold for how many people have to do this to make it worse?
3. Can we solve this problem through simulation and probability (numerical and analytical)?

I don't know much about queuing theory or probability, so this is going to be challenging for me. I will start with the numerical approach

# Dependencies

In [8]:
import simpy
import random
import numpy as np

## Simplifying Assumption
- elevator going down picks up people that it can pick up (regardless of queue)
- if someone gets on at a non lobby floor, they're going to the lobby

In [34]:
a = [(1,2), (3,4), (5,6)]
list(map(list, zip(*a)))
a + a

[(1, 2), (3, 4), (5, 6), (1, 2), (3, 4), (5, 6)]

In [19]:
class elevator(object):
    def __init__(self, name):
        self.name = name
        self.loc = 0
        self.moving_up = True
        self.to_pick_up = []
        self.to_drop_of = []
        self.customers = []
        self.stopped = 0
        self.idle = True
    def move(self):
        if self.idle:
            print('idle elevator')
        elif self.stopped != 0:
            # if stopped, stay stopped
            self.stopped -= 1
        elif #logic for should I stop:

            # if should stop, stop
            # pick up and drop off as necessary
            self.stop()
        elif (self.empty and len(self.to_pick_up) == 0):
            self.idle = True
            print('switching to idle') 
        else:
            self.loc += .5 if self.moving_up else -.5
            #continue moving
                
    def pick_up_and_drop_off(self):
        # picking people up
        for customer in self.to_pick_up:
            if customer.curr == self.loc:
                self.to_drop_off.enqueue(customer)
                self.to_pick_up.remove(customer)
                
        # dropping people off
        for customer in self.to_drop_off:
            if customer.dest == self.loc:
                self.to_drop_off.remove(customer)
        
        # which direction to proceed?
        if moving_up:
            # check if anyone is above me
            for customer in self.to_pick_up:
                if customer.curr > self.loc:
                    return # continue moving up
            for customer in self.to_drop_off:
                if customer.dest > self.loc:
                    return # continue moving up
            # no one above me, turn around
            moving_up = False
        else:
            # check if anyone is above me
            for customer in self.to_pick_up:
                if customer.curr < self.loc:
                    return # continue moving up
            for customer in self.to_drop_off:
                if customer.dest < self.loc:
                    return # continue moving up
            # no one above me, turn around
            moving_up = True
            
            # Could simplify this by having customers only have a destination, and then internalize the logic of switching to object

                        
    def stop(self):
        self.stopped = WAIT_TIME
        self.queue = list(filter(lambda x: x != self.loc, self.queue))
        if len(self.queue) == 0:
            self.empty = True
    
    def enqueue_customer(self, customer):
        self.to_pick_up.append(customer)
            
class customer(object):
    def __init(self):
        self.number = cust_number
        cust_number += 1
        rand = np.random.random()
        randint = np.random.randint(3,23)
        # half the time, someone gets on at the lobby, other half someone gets on a random floor
        self.curr, self.dest = (0, np.random.randint(3,23)) if rand <= .5 else (np.random.randint(3,23), 0) 

WAIT_TIME = 5
time_steps = 3600 # number of seconds in simulation (one hour)
e1_queue = [] #queue for e1
e1_loc = 0 #curloc
e1_empty = True
e2_queue = [] 
e2_loc = 0 #curloc
e2_empty = True
# person calling an elevator goes into queue as (id, current floor, destination floor)
n_floors = 23 
poisson_lambda = 20 #number of seconds that pass before someone demands an elevator
sec_p_floor = 2 # seconds per floor (without someone there)
sec_stop = 5 #time it takes to stop
arrivals = np.random.poisson(lam=poisson_lambda, size=1000).cumsum()
cust_number = 0 
    
for i in range(time_steps):
    if i in arrivals:
        temp = customer()
    move()

customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arrived
customer arriv